# Annotated directions analysis

In [4]:
import os
import tqdm

## 1 &emsp; Preparation

Gathering paths to plays' `.csv` files and their IDs (which are basically author's name and play title):

In [5]:
path_to_annot = "./data/stage/annotated/"
plays_csv = [play_id for play_id in os.listdir(path_to_annot) if play_id != ".DS_Store"]
play_paths = [path_to_annot + play_id for play_id in plays_csv]
play_ids = [play_csv[:-4] for play_csv in plays_csv]

### 1.1 &emsp; Data not present in .csv

Additional functions: adding year of the play and `play_id`:

In [6]:
import requests
import json

Extracting **year of the play**, fetched from corpus metadata:

In [7]:
request_link = "https://dracor.org/api/corpora/rus/metadata"
response = requests.get(request_link)
if response.status_code == 200:
    play_metadata = response.json()

In [12]:
def append_play_year(play_id, play_metadata):
    """Using metadata from DraCor API, extracts yearNormalized
    for the play.
    
    Calculation of yearNormalized:
    > if date_print and date_premiere:
    >     yearNormalized = min(date_print, date_premiere)
    > elif date_premiere:
    >     yearNormalized = date_premiere
    > else:
    >     yearNormalized = date_print
    > if date_written and yearNormalized:
    >     if yearNormalized - date_written > 10:
    >         yearNormalized = date_written
    >     elif date_written and not yearNormalized:
    >         yearNormalized = date_written
    
    :args play_id (str) — author's name and title of the play
    play_metadata (json) — metadata for all the plays currently
    present in the corpus
    
    :returns year (int) — normalized year of the play
    """
    
    for play in play_metadata:
        if play["playName"] == play_id:
            year = int(play["yearNormalized"])
            break
        else:
            year = 0
    if year == 0:
        print("Couldn't retrieve year.\n\t- play: {}\n\t- status code: {} \
            \n=> Returning 0".format(play_id, response.status_code))
    return year

## 2 &emsp; Assembling everything together

In [15]:
import pandas as pd

In [19]:
def csv_to_full_df(play_id):
    """Converts .csv files for /data/stage/annotated into a 
    pd.DataFrame, then adds 
        (1) directions' origin, 
        (2) normalized year of the play (see append_play_year()).
            
    :args play_id (str) — author's name and title of the play
    
    :returns play_df (pd.DataFrame) — converted data of a single play
    """
    
    play_csv_path = "./data/stage/annotated/{}.csv".format(play_id)
    play_df = pd.read_csv(play_csv_path, sep=";", encoding="utf-8").fillna(0)
    
    # convert direction marks to categories
    for dir_type in play_df.columns[1:]:
        play_df[dir_type] = play_df[dir_type].astype("category")
    
    # add play_id so that we know directions' origin
    add_play_name = lambda play_id : play_id
    play_df["play"] = [add_play_name(play_id)] * len(play_df)
    
    # add yearNormalized
    play_df["year"] = [append_play_year(play_id, play_metadata)] * len(play_df)
    
    return play_df

Demo check:

In [21]:
csv_to_full_df("lermontov-maskarad").head()

,text,setting,entrance,exit,business,delivery,modifier,location,unknown,play,year
0,"Игроки, князь Звездич, Казарин и Шприх. За сто...",1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,lermontov-maskarad,1842
1,(тихо первому),0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,lermontov-maskarad,1842
2,(насмешливо),0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,lermontov-maskarad,1842
3,"(сквозь зубы, уходя)",0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,lermontov-maskarad,1842
4,"Князь, выпив стакан лимонаду, садится к сторон...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,lermontov-maskarad,1842


Concatenating all plays in a single dataset:

In [29]:
df_total = pd.DataFrame(columns=["text", "setting", "entrance", "exit", "business", "delivery", 
                                 "modifier", "location", "unknown", "play", "year"])

In [32]:
for play_id in play_ids:
    play_df = csv_to_full_df(play_id)
    df_total = pd.concat([df_total, play_df], ignore_index=True)
    # except:
    #   print("Error at play: {}".format(play_id))

Checking that everything is OK:

In [33]:
df_total.head()

,text,setting,entrance,exit,business,delivery,modifier,location,unknown,play,year
0,"Игроки, князь Звездич, Казарин и Шприх. За сто...",1,0,0,1,0,0,0,0,lermontov-maskarad,1842
1,(тихо первому),0,0,0,0,1,0,0,0,lermontov-maskarad,1842
2,(насмешливо),0,0,0,0,1,0,0,0,lermontov-maskarad,1842
3,"(сквозь зубы, уходя)",0,0,1,0,1,0,0,0,lermontov-maskarad,1842
4,"Князь, выпив стакан лимонаду, садится к сторон...",0,0,0,1,0,0,0,0,lermontov-maskarad,1842


In [42]:
df_total[df_total["play"] == "krylov-urok-dochkam"].head()

,text,setting,entrance,exit,business,delivery,modifier,location,unknown,play,year
401,"Даша, Семен и потом Лиза.",0,1,0,0,0,0,0,0,krylov-urok-dochkam,1807
402,(выглядывая из другой комнаты.),0,0,0,1,0,0,0,0,krylov-urok-dochkam,1807
403,(показываясь.),0,0,0,1,0,0,0,0,krylov-urok-dochkam,1807
404,(Уходит.),0,0,1,0,0,0,0,0,krylov-urok-dochkam,1807
405,(почесывая лоб.),0,0,0,1,0,0,0,0,krylov-urok-dochkam,1807


In [34]:
len(df_total)

6541

In [40]:
df_total["play"].unique()

array(['lermontov-maskarad', 'krylov-urok-dochkam', 'tolstoy-tsar-boris',
       'griboyedov-gore-ot-uma', 'chekhov-tri-sestry',
       'ostrovsky-svoi-ljudi', 'chekhov-djadja-vanja',
       'fonvizin-nedorosl', 'chekhov-chaika', 'sumarokov-horev',
       'ostrovsky-bespridannitsa', 'chekhov-vishnevyi-sad',
       'ostrovsky-les', 'tolstoy-zhivoy-trup', 'turgenev-holostjak',
       'gogol-zhenitba', 'gogol-revizor', 'pushkin-boris-godunov'],
      dtype=object)

## 3 &emsp; Saving

This data is raw — we haven't done any analysis yet. Nevertheless, it might be useful to save it for later research.

In [43]:
df_total.to_csv("./data/csv/annot_merged_raw.csv", sep=";", encoding="utf-8", index=False)